In [1]:
import pandas as pd
import re

In [2]:
base1000_df = pd.read_csv('data/accre-jobs-2020.csv', nrows=1000)
base1000_df.shape

(1000, 13)

In [3]:
base1000_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   JOBID      1000 non-null   object
 1   ACCOUNT    1000 non-null   object
 2   USER       1000 non-null   object
 3   REQMEM     1000 non-null   object
 4   USEDMEM    1000 non-null   object
 5   REQTIME    1000 non-null   object
 6   USEDTIME   1000 non-null   object
 7   NODES      1000 non-null   int64 
 8   CPUS       1000 non-null   int64 
 9   PARTITION  1000 non-null   object
 10  EXITCODE   1000 non-null   object
 11  STATE      1000 non-null   object
 12  NODELIST   1000 non-null   object
dtypes: int64(2), object(11)
memory usage: 101.7+ KB


In [4]:
base1000_df = base1000_df[base1000_df['PARTITION']=='production']

In [5]:
base1000_df.shape

(991, 13)

In [6]:
base1000_df = base1000_df[base1000_df['EXITCODE']=='0:0']

In [7]:
base1000_df.shape

(991, 13)

In [8]:
base1000_df['USEDTIME']

0      13-18:00:28
1      12-14:50:56
2      13-18:00:20
3      12-03:50:32
4      13-18:00:03
          ...     
995     7-17:16:46
996     7-03:03:50
997     7-09:59:09
998     7-12:40:23
999     7-06:27:41
Name: USEDTIME, Length: 991, dtype: object

In [30]:
point1 = base1000_df.loc[271,'USEDTIME']

In [31]:
point1

'23:35:01'

In [32]:
re.findall('\d{2}:\d{2}:\d{2}', point1)

['23:35:01']

In [33]:
base1000_df['HHMMSS_USED'] = base1000_df['USEDTIME'].str.extract('(\d{2}:\d{2}:\d{2})')

In [35]:
base1000_df.loc[271,'HHMMSS_USED']

'23:35:01'

In [40]:
point2 = base1000_df.loc[999,'USEDTIME']

In [41]:
re.findall('(\d+)-', point2)

['7']

In [38]:
base1000_df['DAYS_USED'] = base1000_df['USEDTIME'].str.extract('(\d+)-')

In [39]:
base1000_df.loc[271,'DAYS_USED']

nan

In [14]:
used_time = base1000_df['USEDTIME'].str.split('-',n=1,expand=True)

In [15]:
base1000_df['DAYS_USED'] = used_time[0]

In [16]:
base1000_df['HHMMSS_USED'] = used_time[1]

In [17]:
base1000_df

,JOBID,ACCOUNT,USER,REQMEM,USEDMEM,REQTIME,USEDTIME,NODES,CPUS,PARTITION,EXITCODE,STATE,NODELIST,HHMMSS_USED,DAYS_USED
0,15925210,treviso,arabella,122880Mn,65973.49M,13-18:00:00,13-18:00:28,1,24,production,0:0,COMPLETED,cn1531,18:00:28,13
1,15861126,treviso,arabella,122880Mn,67181.12M,13-18:00:00,12-14:50:56,1,24,production,0:0,COMPLETED,cn1441,14:50:56,12
2,15861125,treviso,arabella,122880Mn,69111.86M,13-18:00:00,13-18:00:20,1,24,production,0:0,COMPLETED,cn1464,18:00:20,13
3,16251645,treviso,arabella,122880Mn,65317.33M,13-18:00:00,12-03:50:32,1,24,production,0:0,COMPLETED,cn1473,03:50:32,12
4,16251646,treviso,arabella,122880Mn,65876.11M,13-18:00:00,13-18:00:03,1,24,production,0:0,COMPLETED,cn1440,18:00:03,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,17016721_16,sunburst,darlyne,8192Mn,2547.97M,14-00:00:00,7-17:16:46,1,1,production,0:0,COMPLETED,cn322,17:16:46,7
996,17016721_17,sunburst,darlyne,8192Mn,1950.11M,14-00:00:00,7-03:03:50,1,1,production,0:0,COMPLETED,cn322,03:03:50,7
997,17016721_18,sunburst,darlyne,8192Mn,2450.23M,14-00:00:00,7-09:59:09,1,1,production,0:0,COMPLETED,cn322,09:59:09,7
998,17016721_19,sunburst,darlyne,8192Mn,2743.43M,14-00:00:00,7-12:40:23,1,1,production,0:0,COMPLETED,cn322,12:40:23,7


In [18]:
req_time = base1000_df['REQTIME'].str.split('-',n=1,expand=True)

In [19]:
base1000_df['DAYS_REQ'] = used_time[0]

In [20]:
base1000_df['HHMMSS_REQ'] = used_time[1]

In [21]:
base1000_df[['day', 'time']] = base1000_df['USEDTIME'].str.split('-').apply(pd.Series)
base1000_df['datetime'] = pd.to_datetime(base1000_df['time'])
base1000_df['day'] = base1000_df['day'].astype(int)

ValueError: invalid literal for int() with base 10: '23:35:01'

In [ ]:
base1000_df['hour'] = base1000_df['datetime'].dt.hour
base1000_df['minute'] = base1000_df['datetime'].dt.minute
base1000_df['second'] = base1000_df['datetime'].dt.second